In [ ]:
import numpy as np
import wfdb
import glob
import matplotlib.pyplot as plt
from matplotlib.image import imread
import pywt
import pywt.data
import os
from pathlib import Path
from PIL import Image,ImageEnhance
from scipy import fftpack
import shutil
import pandas as pd
import random
import scipy.signal as signal
from wfdb import processing
from itertools import combinations
#from scipy.signal.filter_design import zpk2sos

Extract Lead v1 from the 12-Leads ECG Dataset

In [ ]:
def Get_LeadV1_Numpy():

    for myloop in range(21000, 22000, 1):
        five_digit_value = '%05d' % myloop
        signals, fields = wfdb.rdsamp("<<<Souce Path>>>" + five_digit_value + "_lr", channels=[6],sampto=1000)

        signals = signals.tolist()
        # print(signals)

        newest = [i[0] for i in signals]
        X = [i for i in newest if i != 0.0]
        X = np.array(X)
        np.save("<<<Destination Path>>>" + five_digit_value + '.npy', X)  # save

Get_LeadV1_Numpy()

Numpy Files distruction to classes (with the help of CSV file)

In [ ]:
def distribute_images():

    df = pd.read_csv('<<<ECGs CSV File path>>>')

    for i in range(0, 21838, 1):

        filee_name = df.iloc[i][0]
        print(filee_name)
        five_digit_value = '%05d' % filee_name
        file_name = five_digit_value + ".png"

        class_name = df.iloc[i][12]

        source_dir = "<<<Souce Path>>>" + file_name
        search_dir = '<<<Dest Path>>>' + class_name + '/'
        dest_dir = search_dir

        if not os.path.exists(source_dir):
            print(source_dir + " not found.")
        else:
            if not os.path.exists(search_dir):
                print(source_dir)
                os.makedirs(search_dir)
                shutil.move(source_dir, dest_dir)

            else:
                shutil.move(source_dir, dest_dir)

distribute_images()

Denoising Numpy Files

In [ ]:
def Numpy_Denoise():

    def madev(d, axis=None):
        """ Mean absolute deviation of a signal """
        return np.mean(np.absolute(d - np.mean(d, axis)), axis)

    def wavelet_denoising(x, wavelet='bior3.1', level=1):
        coeff = pywt.wavedec(x, wavelet, mode="per")
        sigma = (1 / 0.6745) * madev(coeff[-level])
        uthresh = sigma * np.sqrt(2 * np.log(len(x)))
        coeff[1:] = (pywt.threshold(i, value=uthresh, mode='hard') for i in coeff[1:])
        return pywt.waverec(coeff, wavelet, mode='per')

    # 20446
    src_dir = "<<<Source Path>>>"
    dest_dir = "<<<Destination Path>>>"

    for file in glob.iglob(src_dir + '**/*.npy', recursive=True):
        existing_file_name = Path(file).stem
        new_file_name = existing_file_name[:5]

        signal = np.load(file)

        # wavelist=['bior1.3','bior2.8','bior3.1','bior3.9','db2','db8','rbio1.5','sym6','sym8']
        wavelist = ['bior3.1']


        for wav in wavelist:
            filtered = wavelet_denoising(signal, wavelet=wav, level=1)
            np.save(dest_dir+new_file_name+'.npy', filtered) # save

Numpy_Denoise()

Frequency Filtration to reduce data size

In [ ]:
def Freq_Filtration():
    #### https://scipy-lectures.org/intro/scipy/auto_examples/plot_fftpack.html

    src_dir = "<<<Source Path>>>"
    dest_dir = "<<<Destination Path>>>"

    for file in glob.iglob(src_dir + '**/*.npy', recursive=True):

        #Extracting File Name
        existing_file_name = Path(file).stem
        new_file_name = existing_file_name[:7]

        #Load the file
        x = np.load(file)  # 17876_lr

        # The FFT of the signal
        sig_fft = fftpack.fft(x)

        # The corresponding frequencies
        sample_freq = fftpack.fftfreq(x.size, d=0.02)

        #The original signal copy
        high_freq_fft = sig_fft.copy()
        high_freq_fft = np.delete(high_freq_fft, np.where(np.abs(sample_freq > 2)))

        #Taking Inverse of FFT
        filtered_sig = fftpack.ifft(high_freq_fft)
        print(len(filtered_sig))

        #Saving resultant filtered signal into npy
        np.save(dest_dir+new_file_name+".npy", filtered_sig)
    
Freq_Filteration()

Spectrograms Using STFT (Scipy Library)

In [ ]:
def Draw_Spectrogram():
    
    src_dir = "<<<Source Path>>>"
    dest_dir = "<<<Destination Path>>>"


    for file in glob.iglob(src_dir + '**/*.npy', recursive=True):
        # Extracting File Name
        existing_file_name = Path(file).stem
        new_file_name = existing_file_name[:7]   

        #Loading file
        x = np.load(file)   
        print(len(x))

        #applying STFT transformation
        f, t, Zxx = signal.stft(x, nfft=9, nperseg=9, noverlap=5, fs=100, window='hann') #NFFT calculate by N base Log 2 with x=len(x) ##fs=100
        plt.figure(figsize=(1.15, 1.19))   

        #plotting spectrogram
        plt.pcolormesh(t, f, np.abs(Zxx), cmap="gray_r")
        plt.axis("off")
        plt.savefig(dest_dir+ new_file_name + '.png',bbox_inches='tight',pad_inches=0)
        plt.close('all')

Draw_Spectrogram()

Raw Signal Graph

In [ ]:
def DrawGraphs():

    src_dir = "<<<Source Path>>>"  
    dest_dir = "<<<Destination Path>>>"

    for file in glob.iglob(src_dir + '**/*.npy', recursive=True):
        # Extracting File Name
        existing_file_name = Path(file).stem
        new_file_name = existing_file_name[:7]  

        # Loading file
        x = np.load(file)   
        print(len(x))

        plt.figure(figsize=(0.64, 0.64))   
        plt.plot(x, 'gray')
        plt.axis("off")

        # saving plots
        plt.savefig(dest_dir + new_file_name + '.jpeg')
        plt.close('all')

        
DrawGraphs()

Two Data Augmentation Approaches on Spectrogram / Raw Signal Images

In [ ]:
def HorizontalFlip():
    src_dir = "<<<Source Path>>>"
    dest_dir = "<<<Destination Path>>>"

    for file in glob.iglob(src_dir + '**/*.png', recursive=True):
        # Extracting File Name
        existing_file_name = Path(file).stem
        new_file_name = existing_file_name[:5]
        print(file)
        #opening Image and convert into 2-D
        img = Image.open(file)
        img = np.array(img)

        # Flipping images with Numpy
        flipped_img = np.fliplr(img)

        img=Image.fromarray(flipped_img)
        img.save(dest_dir+new_file_name+"_hFlip"+".png")
        plt.close('all')

def SetContrast():
    src_dir = "<<<Source Path>>>"
    dest_dir = "<<<Destination Path>>>"

    for file in glob.iglob(src_dir + '**/*.png', recursive=True):
        # Extracting File Name
        existing_file_name = Path(file).stem
        new_file_name = existing_file_name[:5]

        # read the image
        im = Image.open(file)

        # image brightness enhancer
        enhancer = ImageEnhance.Contrast(im)

        factor = 1.7  # increase contrast
        im_output = enhancer.enhance(factor)
        im_output.save(dest_dir + new_file_name + "_Contrast" + ".png")


HorizontalFlip()
SetContrast()